*DELE CA2 Part A Submission*

<!-- # **CIFAR10: Image Synthesis with GANs** -->
# <span style="color:orange; font-weight:bold;">CIFAR10: Image Synthesis with GANs</span>

|          Name        |      Class    | Admin No. |
|----------------------|---------------|-----------|
| Timothy Chia Kai Lun | DAAA/FT/2B/02 | P2106911  |

**<u>Objectives</u>**

The aim of this assignment will be to research and implement existing GAN architecture and methods to generate new images based on the CIFAR10 dataset.

## <span style="color:orange;">1. About CIFAR10</span>

The CIFAR10 and CIFAR100 datasets are labeled subsets of a much larger Tiny Images dataset consisting of 80 million images (Shah, 2021). The CIFAR10 dataset consists of 60,000 32x32 colour images in 10 classes, with 6000 images per class. There are 50,000 training images and 10,000 test images. The in context of image generation, we are tasked with synthesizing new images based on the CIFAR10 dataset in the RGB colour space.

---

## <span style="color:orange;">2. Project Setup</span>

In this section, I will be importing the necessary packages and dataset needed for this assignment. Then, I will conduct an exploration of the dataset to identify any preprocessing steps needed before defining and training our GAN models. I will also be making use of TensorFlow's [`Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) class to create a custom dataset to feed into the GAN models for training.

### 2.1 Importing Packages and CIFAR10 Dataset

In [6]:
import os
import pickle
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from skimage import exposure
from tensorflow.keras.datasets import cifar10
from IPython import display
from tensorflow.data import Dataset
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, Embedding, \
    Reshape, Flatten, Dropout, BatchNormalization, ReLU, LeakyReLU, MaxPooling2D, Concatenate
from tensorflow.keras.metrics import Mean, BinaryAccuracy
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import plot_model

In [3]:
# set matplotlib style
sns.set(rc={'figure.dpi': 120})
sns.set_style('whitegrid')

# set gpu memory growth
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# set random seed
tf.random.set_seed(42)
np.random.seed(42)

### 2.2 Exploratory Data Analysis

From importing the CIFAR10 dataset, we can see that the images come in two sets, one for training and one for testing. The dataset comes in the form of numpy arrays of sizes (50000, 32, 32, 3) and (10000, 32, 32, 3) for train and test sets respectively. I have also verified that there are only 10 classes for this dataset.

In [4]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
n_labels = len(np.unique(train_labels))

print(f'Train image shape: {train_images.shape}')
print(f'Test image shape: {test_images.shape}')
print(f'Number of labels: {n_labels}')

Train image shape: (50000, 32, 32, 3)
Test image shape: (10000, 32, 32, 3)
Number of labels: 10


#### 2.2.1 Inspecting Images

It is important to view what kind of images we are trying to generate new samples from, as certain images may hinder our generative model from producing proper samples. It can be seen from each class that they have some similarity in terms of features for example, airplane images contain largely single coloured backgrounds and have pointy/cylindrical shapes. 

One thing that I notice is that colours of frogs in the dataset tend to blend in with the background and that these biological features may have issues being learnt by our model.

In [ ]:
label_map = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

def display_images(images, labels, n_images=10):
    fig, axes = plt.subplots(nrows=n_labels, ncols=n_images, figsize=(20, 20))
    for i in range(n_images):
        for j in range(n_labels):
            axes[j, i].imshow(images[labels.flatten() == j][i])
            axes[j, i].set_title(label_map[j])
            axes[j, i].axis('off')
    plt.tight_layout()
    plt.show()

display_images(train_images, train_labels)

![caifar10-images](https://i.imgur.com/RIS3n0j.png)

#### 2.2.2 Class Colour Distributions

Our models see these images differently from humans, they only understand the numbers, specifically they "see" these images as numbers/pixel values. Our model will take in training examples and noise from some distribution to learn a transformation to the data distribution. Hence, viewing each classes distribution can help us undetand and visualize what our model will be trying to learn.

In [ ]:
bins = 32

fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20, 5), sharex=True, sharey=True)

for i, ax in zip(range(n_labels), axes.flat):
    idx = np.where(train_labels == i)[0]
    ax.hist(train_images[idx, ..., 0].ravel(), bins=bins, color='r', alpha=.7)
    ax.hist(train_images[idx, ..., 1].ravel(), bins=bins, color='g', alpha=.7)
    ax.hist(train_images[idx, ..., 2].ravel(), bins=bins, color='b', alpha=.7)
    ax.set_title(label_map[i])

fig.legend(['Red', 'Green', 'Blue'], loc='upper right', fontsize=12, ncol=3, bbox_to_anchor=(0.592, 1.0), frameon=False)
fig.suptitle('RGB Distribution by Class', fontsize=16, y=1.05)
plt.tight_layout()
plt.show()

![class-rgb-distribution](https://i.imgur.com/rPbXW2R.png)

### 2.3 Data Preparation

#### 2.3.1 Combining Train and Test Sets

Because of the unsupervised component of GANs, we are not trying to predict a label associated with the data and we are not trying to generalize any kind of predictions to new data. Rather, we are trying to learn some hidden/underlying structure of the data to generate new samples. Hence, I will be combining both sets to be used in training.

In [ ]:
# concate train and test numpy arrays
images = np.concatenate((train_images, test_images), axis=0)
labels = np.concatenate((train_labels, test_labels), axis=0)

#### 2.3.2 Removing Low Contrast Images

As mentioned before, certain images may affect our models ability to generate plausible images. One way I have decided to tackle the this issue is to remove images that that blends light and dark areas which makes them look more flat/soft (Skylum, 2023). 

A similarity we can observe from the subset of low-constrast images below is that they blend in with the background easily, making the subjects indistinguishable and hard to make out. These images might cause the generated image of one class to look like the others or nothing at all, for example, a tiny bird may look like a airplane that is very far away in the image. 

Hence, I will be removing these low-contrasted images and keeping only images with distinguishable subjects. I do this by using scikit-image's [`exposure`](https://scikit-image.org/docs/stable/api/skimage.exposure.html) module to identify images with less that the fraction threshold of 0.15.

In [ ]:
low_contrast_idx = []

# loop over images and check for low contrast
for idx, image in enumerate(images):
    if exposure.is_low_contrast(image, fraction_threshold=0.15):
        low_contrast_idx.append(idx)

# plot images in the low contrast list
fig, axes = plt.subplots(10, 10, figsize=(20, 20))
axes = axes.flatten()

# subset of low contrast images
for i in range(100):
    img_lbl = label_map[labels[low_contrast_idx[i]][0]]
    axes[i].imshow(images[low_contrast_idx[i]])
    axes[i].set_title(f'{img_lbl}\nIndex: {low_contrast_idx[i]}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# delete low contrast images
images = np.delete(images, low_contrast_idx, axis=0)
labels = np.delete(labels, low_contrast_idx, axis=0)

![low-contrast-images](https://i.imgur.com/ieEyP49.png)

### 2.4 Converting to TensorFlow Dataset

In the last step of data preparation, I convert the dataset from a batch of 60,000 numpy arrays to a 4D tensor of shape `(batch_size, height, width, channels)` and `(batch_size, label)` for images and class labels respectively. Finally, I make use of TensorFlow's [`Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) class to apply scaling of the image tensors to the hyperbolic tangent activation function [-1, 1] as was done in the original DCGAN paper (Radford et al., 2016).

In [ ]:
BUFFER_SIZE = 10_000
BATCH_SIZE = 128

dataset = Dataset.from_tensor_slices((images, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 127.5 - 1, y))
image_spec, label_spec = dataset.element_spec 

---

## <span style="color:orange;">3. GAN Intuition</span>

First proposed by Ia Goodfellow, Generative Adversarial Networks (GANs) are a type of generative model that create new data instances that resemble the training data (Goodfellow et al., 2014). 

> "The generative model can be thought of as analogous to a team of counterfeiters, trying to produce fake currency and use it without detection, while the discriminative model is analogous to the police, trying to detect the counterfeit currency. Competition in this game drives both teams to improve their methods until the counterfeits are indistiguishable from the genuine articles."

Which in this case, will be images from CIFAR10. A GAN consists of two networks pitted against each other: a generator that captures the joint probability $p(X,Y)$; a discriminator that captures the conditional probability $p(Y|X)$.

![](https://i.imgur.com/Dy0M5P5.png)

*Credit: [Background: What is a Generative Model?](https://developers.google.com/machine-learning/gan/generative), Google*

In the context of image generation, the generator's goal is to generate images close/similar to the data distribution of the training data such that it fools the discriminator. The discriminator's goal is much simpler, draw boundaries in the data space to be able to seperate real from fake/generated images.

---

## <span style="color:orange;">4. Experiments</span>

In this section, I will go through the different approaches I will be taking to generate new images. I will start with a baseline model, then experiment with different mdethods to further improve the models generative ability. 

### 4.1 Baseline: Conditional DCGAN

DCGAN was first proposed in [Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/abs/1511.06434) which uses CNN architecture for image generation. I will be implementing this architecture with the generator and discriminator networks conditioned on extra information such as class labels. 

![](https://i.imgur.com/v7d9ccH.png)

*DCGAN Architecture (Radford et al., 2016)*

<!-- It employed the following innovations:

- Instead of using fixed pooling operations like Max Pooling in the convolutional blocks, strided convolutions are used, where the convolutional layers can learn their own downsampling operations. Transposed convolutions (also known as fractionally strided convolutions) are used for upsampling.

- Batch normalization which normalizes the inputs to layers to have zero mean and variance of one helps to stabilize learning in deep convolutional layers. However, it was found that applying batch norm to the output layer of the generator and input to the discriminator caused instability in training, so batch norm is omitted for these layers

- Use of ReLU activation in the generator for all layers except the output, which uses the Tanh activation (which restricts the image output to a range of -1 to 1)

- LeakyReLU activation in the discriminator.


While this paper introduced the architecture for unconditional generation, I will adapt it to work for conditional image generation. -->

Auxiliary information such as the class labels can be embedded into the latent vector that is then passed to the generator network. Class information is also embedded into real image samples when fed to the discriminator during training.

![](https://i.imgur.com/dwMuM6E.png)

*Condtional GAN (Mirza & Osindero, 2014)*



#### 4.1.1 Minimax Loss Function

The objective function for a DCGAN would be:

$$
\underset{G}{\operatorname{min}}\space\underset{D}{\operatorname{max}}\space V(D,G)= E_x[log(D(x))] + E_z[log(1-D(G(z)))]
$$

where, $D(x)$ is the discriminator's prediction of the probability that a real image instance is real, $E_x$ is the expected value over all real image instances, $G(z)$ is the generator's output (generated image) given noise from a some distribution, $D(G(z))$ is the discriminator's prediction of the probability that a fake image instance is real, $E_z$ is the expected value over all generated fake instances. The generator aims to minimize the function, while the discriminator aims to maximize it.

However, because we are conditioning both networks to auxiliary information $y$, the loss function becomes:

$$
\underset{G}{\operatorname{min}}\space\underset{D}{\operatorname{max}}\space V(D,G)=E_x[logD(x|y)] + E_z[log(1-D(G(z|y)))]
$$

#### 4.1.2 Generator Network

The generator embeds label information by concatenating inputs from a latent vector $z$ and class labels. This implementation is modified from the [Conditional GAN](https://keras.io/examples/generative/conditional_gan/) example found on Keras documentation. I have also implemented the best practices recommended in [Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/abs/1511.06434) (Radford et al., 2016). Which state to use:

- Strided convolutional layers for discriminator and generator networks.

- ReLU activation in generator for all layers and Tanh in output layer.

- LeakyReLU activation in discriminator for all layers.

In [ ]:
def create_generator(latent_dim):
    # foundation for label embedded input
    label_input = Input(shape=(1,), name='label_input')
    label_embedding = Embedding(10, 10, name='label_embedding')(label_input)
    
    # linear activation
    label_embedding = Dense(4 * 4, name='label_dense')(label_embedding)

    # reshape to additional channel
    label_embedding = Reshape((4, 4, 1), name='label_reshape')(label_embedding)
    assert label_embedding.shape == (None, 4, 4, 1)

    # foundation for 4x4 image input
    noise_input = Input(shape=(latent_dim,), name='noise_input')
    noise_dense = Dense(4 * 4 * 128, name='noise_dense')(noise_input)
    noise_dense = ReLU(name='noise_relu')(noise_dense)
    noise_reshape = Reshape((4, 4, 128), name='noise_reshape')(noise_dense)
    assert noise_reshape.shape == (None, 4, 4, 128)

    # concatenate label embedding and image to produce 129-channel output
    concat = Concatenate(name='concatenate')([noise_reshape, label_embedding])
    assert concat.shape == (None, 4, 4, 129)

    # upsample to 8x8
    conv1 = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', name='conv1')(concat)
    assert conv1.shape == (None, 8, 8, 128)
    conv1 = ReLU(name='conv1_relu')(conv1)

    # upsample to 16x16
    conv2 = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', name='conv2')(conv1)
    assert conv2.shape == (None, 16, 16, 128)
    conv2 = ReLU(name='conv2_relu')(conv2)

    # upsample to 32x32
    conv3 = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', name='conv3')(conv2)
    assert conv3.shape == (None, 32, 32, 128)
    conv3 = ReLU(name='conv3_relu')(conv3)

    # output 32x32x3
    output = Conv2D(3, (3, 3), activation='tanh', padding='same', name='output')(conv3)
    assert output.shape == (None, 32, 32, 3)

    model = Model(inputs=[noise_input, label_input], outputs=output, name='generator')

    return model

#### 4.1.3 Discriminator Network

Similar to the generator network, label information is also embedded into real image instances to then me discriminated by the network during training. As recommended in the DCGAN paper, I used a LeakyReLU activation with an alpha of 0.2.

In [ ]:
def create_discriminator():
    # foundation for label embeedded input
    label_input = Input(shape=(1,), name='label_input')
    label_embedding = Embedding(10, 10, name='label_embedding')(label_input)
    
    # linear activation
    label_embedding = Dense(32 * 32, name='label_dense')(label_embedding)
    
    # reshape to additional channel
    label_embedding = Reshape((32, 32, 1), name='label_reshape')(label_embedding)
    assert label_embedding.shape == (None, 32, 32, 1)

    # foundation for 32x32 image input
    image_input = Input(shape=(32, 32, 3), name='image_input')

    # concatenate label embedding and image to produce 129-channel input
    concat = Concatenate(name='concatenate')([image_input, label_embedding])
    assert concat.shape == (None, 32, 32, 4)

    # downsample to 16x16
    conv1 = Conv2D(128, kernel_size=3, strides=2, padding='same', name='conv1')(concat)
    assert conv1.shape == (None, 16, 16, 128)
    conv1 = LeakyReLU(alpha=0.2, name='conv1_leaky_relu')(conv1)
    
    # downsample to 8x8
    conv2 = Conv2D(128, kernel_size=3, strides=2, padding='same', name='conv2')(conv1)
    assert conv2.shape == (None, 8, 8, 128)
    conv2 = LeakyReLU(alpha=0.2, name='conv2_leaky_relu')(conv2)
    
    # downsample to 4x4
    conv3 = Conv2D(128, kernel_size=3, strides=2, padding='same', name='conv3')(conv2)
    assert conv3.shape == (None, 4, 4, 128)
    conv3 = LeakyReLU(alpha=0.2, name='conv3_leaky_relu')(conv3)

    # flatten feature maps
    flat = Flatten(name='flatten')(conv3)
    
    output = Dense(units=1, activation='sigmoid', name='output')(flat)

    model = Model(inputs=[image_input, label_input], outputs=output, name='discriminator')

    return model

#### 4.1.4 Training Procedure

In [ ]:
class ConditionalDCGAN(Model):
    def __init__(self, generator, discriminator, latent_dim):
        super(ConditionalDCGAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalDCGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.g_loss_metric = Mean(name='g_loss')
        self.d_real_loss_metric = Mean(name='d_real_loss')
        self.d_fake_loss_metric = Mean(name='d_fake_loss')
        self.d_acc_metric = BinaryAccuracy(name='d_acc')

    @property
    def metrics(self):
        return [self.g_loss_metric, self.d_real_loss_metric, self.d_fake_loss_metric, self.d_acc_metric]

    def train_step(self, data):
        real_images, class_labels = data
        class_labels = tf.cast(class_labels, 'int32')
        batch_size = tf.shape(real_images)[0]

        # train discriminator
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        fake_labels = tf.zeros((batch_size, 1))
        real_labels = tf.ones((batch_size, 1)) 

        # freeze generator
        self.discriminator.trainable = True
        self.generator.trainable = False
    
        with tf.GradientTape() as disc_tape:
            disc_tape.watch(self.discriminator.trainable_variables)

            generated_images = self.generator([random_latent_vectors, class_labels], training=True)
            real_output = self.discriminator([real_images, class_labels], training=True)
            fake_output = self.discriminator([generated_images, class_labels], training=True)
            
            d_loss_real = self.loss_fn(real_labels, real_output)
            d_loss_fake = self.loss_fn(fake_labels, fake_output)
            d_loss = d_loss_real + d_loss_fake  # log(D(x)) + log(1 - D(G(z)))
        
        disc_grads = disc_tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(disc_grads, self.discriminator.trainable_variables))

        # train the generator
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        misleading_labels = tf.ones((batch_size, 1))

        # freeze discriminator
        self.discriminator.trainable = False
        self.generator.trainable = True

        with tf.GradientTape() as gen_tape:
            gen_tape.watch(self.generator.trainable_variables)

            generated_images = self.generator([random_latent_vectors, class_labels], training=True)
            pred_on_fake = self.discriminator([generated_images, class_labels], training=True)
            
            g_loss = self.loss_fn(misleading_labels, pred_on_fake)  # log(D(G(z)))
        
        gen_grads = gen_tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(gen_grads, self.generator.trainable_variables))

        # update metrics
        self.g_loss_metric.update_state(g_loss)
        self.d_real_loss_metric.update_state(d_loss_real)
        self.d_fake_loss_metric.update_state(d_loss_fake)
        self.d_acc_metric.update_state(real_labels, real_output)

        return {
            'g_loss': self.g_loss_metric.result(),
            'd_real_loss': self.d_real_loss_metric.result(),
            'd_fake_loss': self.d_fake_loss_metric.result(),
            'd_acc': self.d_acc_metric.result()
        }

#### 4.1.5 Hyperparameters

I have set the learning rate and beta_1 parameters for both generator and discriminator networks to 0.0002 and 0.5 respectively. I trained all models on 1000 epochs and with a 1D noise vector of size 128. One way to improve GAN training is to reduce overconfidence in the discriminator network, which can hurt the GANs generative ability (Goodfellow, 2017). I apply label smoothing to the true labels (labels saying an image is real). By setting the smoothing factor to 0.1, we penalize the discriminator for having predictions that go beyond 0.9.

#### 4.1.6 Training Evaluation

After training for 1,000 epochs, we can see in early stages of training that identifiable subjects starting to form, however these images are never clear enough or contain enough features to distinguish between classes. Eventually at the end of the 1,000th epoch, we can see the generator start to fail due to mode collapse as generated images begin the look similar. Mode collapse happens when the generator is always trying to find the one output that seems most plausible to the discriminator. This can be seen from the loss plot below as the discriminator and generator's loss start to diverge after the first few epochs. 

From the discriminator's accuracy plot, we can see that it eventually get good at discriminating betwen real and generated images which explains why the generated images are still unidentifiable, suggesting that it maybe overfitted, which may prompt some regularization techniques to be applied to the discriminator. 

Loss Plot:

![](https://i.imgur.com/pyGzbYI.png)

Accuracy Plot:

![](https://i.imgur.com/oXiODgt.png)

Images after 100 epochs:

![](https://i.imgur.com/cNKs6E3.png)

Images after 200 epochs:

![](https://i.imgur.com/HIVCQit.png)

Images after 300 epochs:

![](https://i.imgur.com/f37hcBG.png)

Images after 400 epochs:

![](https://i.imgur.com/UO9UuTC.png)

Images after 500 epochs:

![](https://i.imgur.com/wCVhmlt.png)

Images after 1000 epochs:

![](https://i.imgur.com/7jKYC33.png)

### 4.3 Spectral Normalization

Spectral normalization is another way to further stabilize GAN training. Specifically, spectral normalization is applied to the discriminator network, which normalization the weights of the network such that it is Lipschitz continuous. This  means that for a given function, the magnitude of its gradient should be at most K for every point (Cosgrove, 2018).

The effect of the discriminator being Lipschitz continuous is that it put a constraint on the gradients by enforcing an upper bound. And as a result, reduces the risk of exploding gradients. In addition, spectral normalization controls the variance of the weights in the discriminator preventing vanishing gradients (Lin et al., 2021).

In [ ]:
def create_discriminator():
    # foundation for label embeedded input
    label_input = Input(shape=(1,), name='label_input')
    label_embedding = Embedding(10, 10, name='label_embedding')(label_input)
    
    # linear activation
    label_embedding = Dense(32 * 32, name='label_dense')(label_embedding)
    
    # reshape to additional channel
    label_embedding = Reshape((32, 32, 1), name='label_reshape')(label_embedding)
    assert label_embedding.shape == (None, 32, 32, 1)

    # foundation for 32x32 image input
    image_input = Input(shape=(32, 32, 3), name='image_input')

    # concatenate label embedding and image to produce 129-channel input
    concat = keras.layers.Concatenate(name='concatenate')([image_input, label_embedding])
    assert concat.shape == (None, 32, 32, 4)

    # downsample to 16x16
    conv1 = SpectralNormalization(Conv2D(128, kernel_size=3, strides=2, padding='same', name='conv1'))
    conv1 = conv1(concat)
    assert conv1.shape == (None, 16, 16, 128)
    conv1 = LeakyReLU(alpha=0.2, name='conv1_leaky_relu')(conv1)
    
    # downsample to 8x8
    conv2 = SpectralNormalization(Conv2D(128, kernel_size=3, strides=2, padding='same', name='conv2'))
    conv2 = conv2(conv1)
    assert conv2.shape == (None, 8, 8, 128)
    conv2 = LeakyReLU(alpha=0.2, name='conv2_leaky_relu')(conv2)
    
    # downsample to 4x4
    conv3 = SpectralNormalization(Conv2D(128, kernel_size=3, strides=2, padding='same', name='conv3'))
    conv3 = conv3(conv2)
    assert conv3.shape == (None, 4, 4, 128)
    conv3 = LeakyReLU(alpha=0.2, name='conv3_leaky_relu')(conv3)

    # flatten feature maps
    flat = Flatten(name='flatten')(conv3)
    
    output = Dense(units=1, activation='sigmoid', name='output')(flat)

    model = Model(inputs=[image_input, label_input], outputs=output, name='discriminator')

    return model

#### 4.3.1 Training Evaluation

We can see from the loss plots of both networks and discriminator accuracy plot that spectral normalization does indeed improve the stability of training. However from inspecting the generated images, although we can make out what some of the images and their classes (especially at earlier epochs), there are still images generated that look nothing like their respective classes. Eventually at 1000 epochs, images start to produce indistinguishable and mode collapsed images with some having a "melting" effect.

Loss Plot:

![](https://i.imgur.com/RMsBWnk.png)

Accuracy Plot:

![](https://i.imgur.com/oSl7VRa.png)

Images after 100 epochs:

![](https://i.imgur.com/qJTj03B.png)

Images after 200 epochs:

![](https://i.imgur.com/ogohRgU.png)

Images after 300 epochs:

![](https://i.imgur.com/H9P5U64.png)

Images after 400 epochs:

![](https://i.imgur.com/5Zfluo8.png)

Images after 500 epochs:

![](https://i.imgur.com/PhcQvBU.png)

Images after 1000 epochs:

![](https://i.imgur.com/GCsGb8H.png)


### 4.4 Auxiliary Classifier GAN

A variant of the GAN called the Auxiliary Classifier GAN (AC-GAN), was proposed to improve and stabilize training by using auxiliary label information similar to conditional GANs. In the paper, the authors explain that their change in structure and specialized loss function resulted in increased image quality (Odena et al., 2017).

Like conditional GANs, the generator outputs an image given a noise vector $z$ and a corresponding class label. The differences in architecture are found in the discriminator where instead of only having a single output neuron with a sigmoid activation to predict an images source (real/fake), it has a second output layer with neurons equal to the number of class labels in the dataset which use a softmax activation function to predict the class of an image instance. In effect, AC-GANs learn a representation for $z$ that is independent of class labels.

![](https://i.imgur.com/HDy0nWK.png)

*Differences in network structure* (Brownlee, 2021)*

In [ ]:
def create_discriminator():
    input_layer = Input(shape=(32, 32, 3), name='image_input')

    # downsample to 16x16
    conv1 = Conv2D(128, kernel_size=3, strides=2, padding='same', name='conv1')(input_layer)
    assert conv1.shape == (None, 16, 16, 128)
    conv1 = LeakyReLU(alpha=0.2, name='conv1_leaky_relu')(conv1)

    # downsample to 8x8
    conv2 = Conv2D(128, kernel_size=3, strides=2, padding='same', name='conv2')(conv1)
    assert conv2.shape == (None, 8, 8, 128)
    conv2 = LeakyReLU(alpha=0.2, name='conv2_leaky_relu')(conv2)

    # downsample to 4x4
    conv3 = Conv2D(128, kernel_size=3, strides=2, padding='same', name='conv3')(conv2)
    assert conv3.shape == (None, 4, 4, 128)
    conv3 = LeakyReLU(alpha=0.2, name='conv3_leaky_relu')(conv3)

    # flatten feature maps
    flat = Flatten(name='flatten')(conv3)

    # output layers
    sigmoid_out = Dense(units=1, activation='sigmoid', name='sigmoid_output')(flat)
    softmax_out = Dense(units=10, activation='softmax', name='softmax_output')(flat)

    model = Model(input_layer, [sigmoid_out, softmax_out], name='discriminator')

    return model

#### 4.4.1 AC-GAN Loss Function

> "$G$ uses both to generate images $X_{fake} = G(c, z)$. The discriminator gives both
a probability distribution over sources and a probability distribution over the class labels, $P(S | X), P(C | X) =
D(X)$. " (Odena et al., 2017)

The objective function for AC-GAN is defined as

$$
L_s = E[log\space P(S=real| X_{real}) + E[log\space P(S={fake}| X_{fake})]]
$$

$$
L_c = E[log\space P(C=c)| X_{real} + E[log\space P(C=c| X_{fake})]]
$$

where $L_s$ is the log-probability of the correct source (real/fake), $L_c$ is the log-probability of the correct class. The discriminator network $D$ is training to maximize $L_s+L_c$ while the generator network $G$ is trained to maximize $L_c-L_s$.

#### 4.4.2 Training Procedure

In [ ]:
class ACGAN(Model):
    def __init__(self, generator, discriminator, latent_dim):
        super(ACGAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_1, loss_2):
        super(ACGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_1 = loss_1 # disc and gen loss
        self.loss_2 = loss_2 # auxiliary loss
        self.g_loss_metric = keras.metrics.Mean(name='g_loss')
        self.d_loss_metric = keras.metrics.Mean(name='d_loss')  # disc bce loss
        self.aux_loss_metric = keras.metrics.Mean(name='aux_loss')  # disc cce loss
        self.d_acc_metric = keras.metrics.BinaryAccuracy(name='d_acc')

    @property
    def metrics(self):
        return [self.g_loss_metric, self.d_loss_metric, self.aux_loss_metric, self.d_acc_metric]

    def train_step(self, data):
        real_images, class_labels = data
        class_labels = tf.cast(class_labels, tf.float32)
        batch_size = tf.shape(real_images)[0]

        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        generated_images = self.generator([random_latent_vectors, class_labels], training=True)

        # freeze generator
        self.discriminator.trainable = True
        self.generator.trainable = False

        with tf.GradientTape() as disc_tape:
            real_pred, real_aux = self.discriminator(real_images, training=True)
            fake_pred, fake_aux = self.discriminator(generated_images, training=True)

            # discriminator loss
            d_loss_1 = self.loss_1(tf.ones_like(real_pred), real_pred)
            d_loss_2 = self.loss_1(tf.zeros_like(fake_pred), fake_pred)
            d_loss = d_loss_1 + d_loss_2
            # auxiliary loss
            aux_loss = self.loss_2(class_labels, fake_aux)
            # total discriminator loss
            d_loss += aux_loss

        # discriminator gradients
        d_grads = disc_tape.gradient(d_loss, self.discriminator.trainable_weights)
        # update discriminator
        self.d_optimizer.apply_gradients(zip(d_grads, self.discriminator.trainable_weights))

        # generator loss
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # freeze discriminator
        self.discriminator.trainable = False
        self.generator.trainable = True

        with tf.GradientTape() as gen_tape:
            generated_images = self.generator([random_latent_vectors, class_labels], training=True)
            fake_pred, fake_aux = self.discriminator(generated_images, training=True)
            g_loss_1 = self.loss_1(tf.ones_like(fake_pred), fake_pred)
            g_loss_2 = self.loss_2(class_labels, fake_aux)
            g_loss = g_loss_1 + g_loss_2

        # generator gradients
        g_grads = gen_tape.gradient(g_loss, self.generator.trainable_weights)
        # update generator
        self.g_optimizer.apply_gradients(zip(g_grads, self.generator.trainable_weights))

        # update metrics
        self.g_loss_metric.update_state(g_loss)
        self.d_loss_metric.update_state(d_loss)
        self.aux_loss_metric.update_state(aux_loss)
        self.d_acc_metric.update_state(tf.ones_like(real_pred), real_pred)
        
        return {
            'g_loss': self.g_loss_metric.result(),
            'd_loss': self.d_loss_metric.result(),
            'aux_loss': self.aux_loss_metric.result(),
            'd_acc': self.d_acc_metric.result()
        }

#### 4.4.3 Training Evaluation

After training the AC-GAN, the loss and accuracy plots indicate that training is definitely more stable than the baseline conditional DCGAN. Images generated by AC-GAN definitely have higher quality as compared to the baseline, however the images do not seem to be representative of their respective classes and are still not as distinguishable as compared the the baseline and using spectral normalization. 
<!-- This may be due to a fault in my implementation. -->

Loss Plot:

![](https://i.imgur.com/EZg34ue.png)

Accuracy Plot:

![](https://i.imgur.com/yZYHVWi.png)

Images after 100 epochs:

![](https://i.imgur.com/XplzwB4.png)

Images after 200 epochs:

![](https://i.imgur.com/mihGoyN.png)

Images after 300 epochs:

![](https://i.imgur.com/3odALMg.png)

Images after 400 epochs:

![](https://i.imgur.com/hqGUFDc.png)

Images after 500 epochs:

![](https://i.imgur.com/cz4HwpN.png)

Images after 1000 epochs:

![](https://i.imgur.com/x5PzCOW.png)

---

## <span style="color:orange;">5. Evaluation</span>

Comparing each models generated images, just with observation I would argue that the base conditional DCGAN is sufficient to generate CIFAR10 images although they are somewhat low quality. The model is able to pick on the features of the images and come up with plausible images. 

![](https://i.imgur.com/UO9UuTC.png)
*Conditional DCGAN at 400 epochs*

One thing I have observed as well is that the loss metrics are not sufficient and not helpful, for example, visually, the images generated by DCGAN at epoch 400 would be able to fool even me. However the diverging losses of the both networks in the DCGAN tell me the discriminator is getting too good at discriminating real/fake images while the generator is getting worse at generating images. Furthermore, the loss plots of DCGAN with spectral normalization and AC-GAN 
show a more stable training, however observing the generated images tells us otherwise.

In hindsight, I would try to use a more informative metric to evaluate my models like the Inception Score (IS), Frechet Inception Distance (FID), and Kernel Inception Distance (KID).

---

## <span style="color:orange;">6. Final Images</span>

With the models I have, I have chosen to use the Conditional DCGAN's trained generator at 400 epochs to generate my final 1000 images. Each class will have 100 generated images each.

Airplane:
![](https://i.imgur.com/NuTI8Ym.png)

Automobile:
![](https://i.imgur.com/7OzV5EE.png)

Bird:
![](https://i.imgur.com/pHNUUAR.png)

Cat:
![](https://i.imgur.com/oivhavg.png)

Deer:
![](https://i.imgur.com/DjdKBxN.png)

Dog:
![](https://i.imgur.com/Ee5Cn6Y.png)

Frog:
![](https://i.imgur.com/GIEiK1p.png)

Horse:
![](https://i.imgur.com/iHdG6iA.png)

Ship:
![](https://i.imgur.com/8IpUPBF.png)

Truck:
![](https://i.imgur.com/yK5FGQZ.png)

---

## <span style="color:orange;">7. Conclusion</span>

- **Animals are harder to generate**:

    We have seen that across each of the 10 classes, that generating inanimate objects such as vehicles are much easier as compared to animals in the dataset. This may be due to vechicles being much more anglular/cuboid shaped (for trucks are automobiles) and have similar features across all real images in the dataset. The intricate details of animals however, appear to be more difficult to learned and generated, with some images only having plausible colours/texture/shapes of the subject but never all of them together. This may be due to the large variation of training examples between each class as animals have different breeds that can have very different features from their broader class labels.

- **Conditional Image Synthesis and Normalization**

    We are able to control the class/properties of the generated image by including class label information during training. Experiments with other GANs that include auxliary information during training such as the AC-GAN have shown to be more stable and hence similar GAN architectures should be explored that provide more control over the class and style of the synthesized images such as the InfoGAN (Chen et al., 2016). Further experiments can be done with normalization techniques on the discriminator such an using instance normalization as spectral normalization has shown to stabilize training.

- **Further Improvements**

    Given more time, I would try experimenting with concepts from reinforment learning, such as using experience replay to supply previously generated images to be discriminated occasionally during training or other stability tricks used in RL (Pfau & Vinyals, 2017). 

---

## <span style="color:orange;">8. References</span>

- Brownlee, J. (2021) How to develop an auxiliary classifier Gan (AC-gan) from scratch with keras, MachineLearningMastery.com. Available at: https://machinelearningmastery.com/how-to-develop-an-auxiliary-classifier-gan-ac-gan-from-scratch-with-keras/ (Accessed: January 29, 2023). 

- Cosgrove, C. (2018) Spectral normalization explained, Ode to Gradients. Available at: https://christiancosgrove.com/blog/2018/01/04/spectral-normalization-explained.html (Accessed: January 29, 2023). 

- Chen, X. et al. (2016) Infogan: Interpretable representation learning by information maximizing generative adversarial nets, arXiv.org. Available at: https://arxiv.org/abs/1606.03657 (Accessed: January 29, 2023). 

- Goodfellow, I. (2017) NIPS 2016 tutorial: Generative Adversarial Networks, arXiv.org. Available at: https://arxiv.org/abs/1701.00160 (Accessed: January 29, 2023). 

- Goodfellow, I.J. et al. (2014) Generative Adversarial Networks, arXiv.org. Available at: https://arxiv.org/abs/1406.2661 (Accessed: January 23, 2023). 

- Krizhevsky, A., Nair, V. and Hinton, G. (no date) CIFAR-10 and CIFAR-100 datasets. Available at: https://www.cs.toronto.edu/~kriz/cifar.html (Accessed: January 21, 2023).

- Lin, Z., Sekar, V. and Fanti, G. (2021) Why spectral normalization stabilizes Gans: Analysis and improvements, arXiv.org. Available at: https://arxiv.org/abs/2009.02773 (Accessed: January 29, 2023).

- Mirza, M. and Osindero, S. (2014) Conditional generative adversarial nets, arXiv.org. Available at: https://arxiv.org/abs/1411.1784 (Accessed: January 29, 2023). 

- Odena, A., Olah, C. and Shlens, J. (2017) Conditional image synthesis with auxiliary classifier Gans, arXiv.org. Available at: https://arxiv.org/abs/1610.09585 (Accessed: January 29, 2023). 

- Pfau, D. and Vinyals, O. (2017) Connecting generative adversarial networks and actor-critic methods, arXiv.org. Available at: https://arxiv.org/abs/1610.01945 (Accessed: January 29, 2023). 

- Radford, A., Metz, L. and Chintala, S. (2016) Unsupervised representation learning with deep convolutional generative Adversarial Networks, arXiv.org. Available at: https://arxiv.org/abs/1511.06434 (Accessed: January 23, 2023). 

- Shah, A. (2021) MIT 80 million Tiny Images dataset, Kaggle. Available at: https://www.kaggle.com/datasets/aryashah2k/mit-80-million-tiny-images-dataset (Accessed: January 21, 2023).

- Understanding contrast in photography (no date) Skylum Blog. Available at: https://skylum.com/blog/understanding-contrast-in-photography (Accessed: January 23, 2023).

In [2]:
!jupyter nbconvert --to html CIFAR10_Image_Synthesis.ipynb  

[NbConvertApp] Converting notebook CIFAR10_Image_Synthesis.ipynb to html
[NbConvertApp] Writing 714115 bytes to CIFAR10_Image_Synthesis.html
